<a href="https://colab.research.google.com/github/dung18520632/Deep_Learning-Python-Basics/blob/main/Planar%20data%20classification%20with%20one%20hidden%20layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from testCases import *
import sklearn
import sklearn.datasets
import sklearn.linear_model
from planar_utils import plot_decision_boundary, sigmoid, load_planar_dataset, load_extra_datasets

In [2]:
X, Y = load_planar_dataset()

In [6]:
print("The shape of X",X.shape)
print("The shape of Y",Y.shape)
m=X.shape[1]


The shape of X (2, 400)
The shape of Y (1, 400)


In [13]:
cfs=sklearn.linear_model.LogisticRegression()
model=cfs.fit(X.T,Y.T)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
Y_prec=model.predict(X.T)
print("Độ chính xác",float((np.dot(Y,Y_prec)+np.dot(1-Y,1-Y_prec)))/float(Y.size)*100)

Độ chính xác 47.0
